In [1]:
# Data loading
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output

from joblib import load
from tqdm import trange
from tqdm.notebook import tqdm

# Graph dataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset, Data

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold


# GNN Model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv, GATConv, GATv2Conv, SAGEConv


# Sparse vector
from Sparse_vector.sparse_vector import SparseVector

In [2]:
from tqdm import tqdm
import sys
import time

tqdm.pandas()

In [3]:
chrom_names = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y','M']]

features = [i[:-4] for i in os.listdir('z_dna/hg38_features/sparse/') if i.endswith('_2.pkl')]
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in features]

In [4]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'z_dna/hg38_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"z_dna/hg38_dna/{file}") for file in files])

In [5]:
%%time
DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chrom_names)}
#ZDNA = load('z_dna/hg38_zdna/sparse/ZDNA_shin.pkl')
#ZDNA = load('z_dna/hg38_zdna/sparse/ZDNA_cousine.pkl')

ZDNA = load('z_dna/hg38_zdna/sparse/ZDNA_cousine.pkl')

DNA_features = {feature: load(f'z_dna/hg38_features/sparse/{feature}.pkl')
                for feature in tqdm(feature_names)}

100%|███████████████████████████████████████████████████████████████████████████████| 1946/1946 [00:57<00:00, 33.56it/s]

CPU times: user 57.5 s, sys: 4.95 s, total: 1min 2s
Wall time: 1min 2s


In [6]:
width = 100



In [7]:
from torch_geometric.data import Dataset, Data
class GraphDataset(Dataset):
    def __init__(self, chroms, features,
                 dna_source, features_source,
                 labels, intervals,
                 transform=None, pre_transform=None, pre_filter=None):
        self.chroms = chroms
        self.features = features
        self.dna_source = dna_source
        self.features_source = features_source
        self.labels = labels
        self.intervals = intervals
        self.le = LabelBinarizer().fit(np.array([["A"], ["C"], ["T"], ["G"]]))

        self.ei = [[],[]]
        for i in range(width-1):
            self.ei[0].append(i)
            self.ei[0].append(i+1)
            self.ei[1].append(i+1)
            self.ei[1].append(i)
        super().__init__(transform, pre_transform, pre_filter)

    def len(self):
        return len(self.intervals)

    def get(self, idx):
        interval = self.intervals[idx]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        dna_OHE = self.le.transform(list(self.dna_source[chrom][begin:end].upper()))

        feature_matr = []
        for feature in self.features:
            source = self.features_source[feature]
            feature_matr.append(source[chrom][begin:end])

        if len(feature_matr) > 0:
            X = np.hstack((dna_OHE, np.array(feature_matr).T/1000)).astype(np.float32)
        else:
            X = dna_OHE.astype(np.float32)
        X = torch.tensor(X, dtype=torch.float)

        edge_index = torch.tensor(np.array(self.ei), dtype=torch.long)

        y = self.labels[interval[0]][interval[1]: interval[2]]
        y = torch.tensor(y, dtype=torch.int64)

        return Data(x=X.unsqueeze(0), edge_index=edge_index, y=y.unsqueeze(0))

In [8]:
np.random.seed(10)
width = 100

ints_in = []
ints_out = []

for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width, width):
        interval = [st, min(st + width, ZDNA[chrm].shape)]
        if ZDNA[chrm][interval[0]: interval[1]].any():
            ints_in.append([chrm, interval[0], interval[1]])
        else:
            ints_out.append([chrm, interval[0], interval[1]])

ints_in = np.array(ints_in)
ints_out = np.array(ints_out)[np.random.choice(range(len(ints_out)), size=len(ints_in) * 2, replace=False)]

100%|██████████████████████████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 37858.87it/s]


In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
np.random.seed(42)
equalized = np.vstack((ints_in, ints_out))
equalized = [[inter[0], int(inter[1]), int(inter[2])] for inter in equalized]

labels = np.array([1]*len(ints_in) + [0]*len(ints_out))
chromes = [inter[0] for inter in equalized]
strat_labels = np.array([f"{label}_{chrom}" for label, chrom in zip(labels, chromes)])
print(strat_labels)
  
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_inds, test_inds = next(sss.split(equalized, strat_labels))
  
train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]

['1_chr1' '1_chr1' '1_chr1' ... '0_chr18' '0_chr8' '0_chr6']


In [10]:
np.random.seed(42)
train_dataset = GraphDataset(chrom_names, feature_names,
                            DNA, DNA_features,
                            ZDNA, train_intervals)

test_dataset = GraphDataset(chrom_names, feature_names,
                           DNA, DNA_features,
                           ZDNA, test_intervals)

In [11]:
np.random.seed(42)
from torch_geometric.loader import DataLoader
params = {'batch_size':32,
          'num_workers':4,
          'shuffle':True}

loader_train = DataLoader(train_dataset, **params)
loader_test = DataLoader(test_dataset, **params)

In [12]:
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [13]:
set_random_seed(42)

# GNN Model

In [14]:
class GraphZSAGEConv_11L(torch.nn.Module):
    def __init__(self):
        super(GraphZSAGEConv_11L, self).__init__()
        
        
        self.conv1 = SAGEConv(1950, 1024)
        self.conv2 = SAGEConv(1024, 1024)
        self.conv3 = SAGEConv(1024, 512)
        self.conv4 = SAGEConv(512, 512)
        self.conv5 = SAGEConv(512, 256)
        
        self.conv6 = SAGEConv(256, 256)
        self.conv7 = SAGEConv(256, 128)
        self.conv8 = SAGEConv(128, 128)
        self.conv9 = SAGEConv(128, 64)
        self.conv10 = SAGEConv(64, 64)
        
        self.conv11 = SAGEConv(64, 2)
        
        self.norm1 = torch.nn.GroupNorm(num_groups=512, num_channels=1024)
        self.norm2 = torch.nn.GroupNorm(num_groups=512, num_channels=1024)
        self.norm3 = torch.nn.GroupNorm(num_groups=256, num_channels=512)
        self.norm4 = torch.nn.GroupNorm(num_groups=256, num_channels=512)
        self.norm5 = torch.nn.GroupNorm(num_groups=128, num_channels=256)
        self.norm6 = torch.nn.GroupNorm(num_groups=128, num_channels=256)
        self.norm7 = torch.nn.GroupNorm(num_groups=64, num_channels=128)
        self.norm8 = torch.nn.GroupNorm(num_groups=64, num_channels=128)
        self.norm9 = torch.nn.GroupNorm(num_groups=32, num_channels=64)
        self.norm10 = torch.nn.GroupNorm(num_groups=32, num_channels=64)

    def forward(self, x, edge):
        
        if x.dim() == 2:
            x = x.unsqueeze(0)
        
        x = self.conv1(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm1(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm2(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        
        x = self.conv3(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm3(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv4(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm4(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv5(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm5(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv6(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm6(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv7(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm7(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv8(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm8(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv9(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm9(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv10(x, edge)
        x = x.permute(0, 2, 1)
        x = self.norm10(x)
        x = x.permute(0, 2, 1)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv11(x, edge)

        return F.log_softmax(x, dim=-1)

In [15]:
model = GraphZSAGEConv_11L()
model.load_state_dict(torch.load("GraphZSAGEConv_11L.pt"))
#model = model.cuda()
model.eval()

GraphZSAGEConv_11L(
  (conv1): SAGEConv(1950, 1024, aggr=mean)
  (conv2): SAGEConv(1024, 1024, aggr=mean)
  (conv3): SAGEConv(1024, 512, aggr=mean)
  (conv4): SAGEConv(512, 512, aggr=mean)
  (conv5): SAGEConv(512, 256, aggr=mean)
  (conv6): SAGEConv(256, 256, aggr=mean)
  (conv7): SAGEConv(256, 128, aggr=mean)
  (conv8): SAGEConv(128, 128, aggr=mean)
  (conv9): SAGEConv(128, 64, aggr=mean)
  (conv10): SAGEConv(64, 64, aggr=mean)
  (conv11): SAGEConv(64, 2, aggr=mean)
  (norm1): GroupNorm(512, 1024, eps=1e-05, affine=True)
  (norm2): GroupNorm(512, 1024, eps=1e-05, affine=True)
  (norm3): GroupNorm(256, 512, eps=1e-05, affine=True)
  (norm4): GroupNorm(256, 512, eps=1e-05, affine=True)
  (norm5): GroupNorm(128, 256, eps=1e-05, affine=True)
  (norm6): GroupNorm(128, 256, eps=1e-05, affine=True)
  (norm7): GroupNorm(64, 128, eps=1e-05, affine=True)
  (norm8): GroupNorm(64, 128, eps=1e-05, affine=True)
  (norm9): GroupNorm(32, 64, eps=1e-05, affine=True)
  (norm10): GroupNorm(32, 64, eps=1

# Captum methods: IntegratedGradients

In [16]:
!pip install captum


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [17]:
import captum
from captum.attr import IntegratedGradients, GradientShap, LayerGradCam, LRP
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from torch_geometric.explain import Explainer, CaptumExplainer

In [29]:
explainer = Explainer(
    model=wrappedmodel,
    algorithm=CaptumExplainer('IntegratedGradients'),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='probs',
    ),
)

np.random.seed(42)
params = {'batch_size':1,
          'num_workers':4,
          'shuffle':True}

loader_train = DataLoader(train_dataset, **params)
loader_test = DataLoader(test_dataset, **params)

In [30]:
features_count = 1950

In [31]:
%%time
features_count =1950
mean_1 = np.zeros(1950, dtype=float)
cnt= 0
cc = 0

device = torch.device(f'cuda:{1}')
with torch.cuda.device(device):
    model = model.to('cuda:1')
    for dt in tqdm(loader_test):
        
        x, edge, y = dt.x.to('cuda:1'), dt.edge_index.to('cuda:1'), dt.y.to('cuda:1').long()
        
        output = model(x, edge)
        pred = torch.argmax(output, dim=-1)
        
        idxs = []
        for i in range(width):
            if pred[0][i] == y[0][i] and y[0][i] == 1:
                idxs.append(i)
        
        torch.cuda.empty_cache()
        explanation = explainer(x.squeeze(), edge, target=1)
        node_mask = explanation.node_mask

        if node_mask[idxs, :].shape != (0, features_count):
            node_mask = torch.mean(node_mask[idxs, :], dim=0)
            node_mask = np.array(node_mask.cpu())
            mean_1 += node_mask
            cnt += 1

print('done interpretation')

  0%|                                                                                         | 0/27121 [00:00<?, ?it/s]/home/alapteva/local/python3.9.0/lib/python3.9/site-packages/torch_geometric/explain/explainer.py:193: UserWarning: The 'target' should not be provided for the explanation type 'model'
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████| 27121/27121 [9:36:40<00:00,  1.28s/it]

done interpretation
CPU times: user 9h 20min 52s, sys: 15min 38s, total: 9h 36min 31s
Wall time: 9h 36min 40s


In [33]:
mean = mean_1 / cnt
print(mean.shape)
print(mean)

(1950,)
[-1.84972169e-01 -6.31486791e-02 -2.29900256e-02 ... -1.79317337e-15
 -5.23944621e-10  9.22915443e-07]


In [34]:
mean

array([-1.84972169e-01, -6.31486791e-02, -2.29900256e-02, ...,
       -1.79317337e-15, -5.23944621e-10,  9.22915443e-07])

In [35]:
torch.save(torch.from_numpy(mean), 'GraphZSAGEConv_11L_IG.pt')

# salency

In [18]:
class WrappedModel(torch.nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, x, edge_index):
        out = self.base_model(x, edge_index) 
        a = out[:, :, 1] 
        return a 

In [19]:
wrappedmodel = WrappedModel(model)

In [129]:
from torch_geometric.explain import Explainer, CaptumExplainer

explainer = Explainer(
    model=wrappedmodel,
    algorithm=CaptumExplainer('Saliency'),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='probs',
    ),
)

In [132]:
np.random.seed(42)
params = {'batch_size':1,
          'num_workers':4,
          'shuffle':True}

loader_train = DataLoader(train_dataset, **params)
loader_test = DataLoader(test_dataset, **params)

In [133]:
%%time
features_count =1950
mean_1 = np.zeros(1950, dtype=float)
cnt= 0
cc = 0

device = torch.device(f'cuda:{1}')
with torch.cuda.device(device):
    model = model.to('cuda:1')
    for dt in tqdm(loader_test):
        
        x, edge, y = dt.x.to('cuda:1'), dt.edge_index.to('cuda:1'), dt.y.to('cuda:1').long()
        
        output = model(x, edge)
        pred = torch.argmax(output, dim=-1)
        
        idxs = []
        for i in range(width):
            if pred[0][i] == y[0][i] and y[0][i] == 1:
                idxs.append(i)
        
        torch.cuda.empty_cache()
        explanation = explainer(x.squeeze(), edge, target=1)
        node_mask = explanation.node_mask

        if node_mask[idxs, :].shape != (0, features_count):
            node_mask = torch.mean(node_mask[idxs, :], dim=0)
            node_mask = np.array(node_mask.cpu())
            mean_1 += node_mask
            cnt += 1

print('done interpretation')

100%|█████████████████████████████████████████████████████████████████████████████| 27121/27121 [36:28<00:00, 12.39it/s]

done interpretation
CPU times: user 31min 43s, sys: 4min 19s, total: 36min 2s
Wall time: 36min 28s


In [135]:
mean = mean_1 / cnt
print(mean.shape)

(1950,)


In [136]:
mean

array([1.76419802e-02, 1.97136785e-02, 2.26526768e-02, ...,
       2.68914499e-09, 5.58110266e-05, 1.61702059e-03])

In [137]:
torch.save(torch.from_numpy(mean), 'GraphZSAGEConv_11L_Saliency.pt')

In [138]:
torch.load('GraphZSAGEConv_11L_Saliency.pt')

tensor([1.7642e-02, 1.9714e-02, 2.2653e-02,  ..., 2.6891e-09, 5.5811e-05,
        1.6170e-03], dtype=torch.float64)

# IxG

In [23]:
from torch_geometric.explain import Explainer, CaptumExplainer

explainer = Explainer(
    model=wrappedmodel,
    algorithm=CaptumExplainer('InputXGradient'),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='probs',
    ),
)

In [24]:
np.random.seed(42)
params = {'batch_size':1,
          'num_workers':4,
          'shuffle':True}

loader_train = DataLoader(train_dataset, **params)
loader_test = DataLoader(test_dataset, **params)

In [25]:
%%time
features_count =1950
mean_1 = np.zeros(1950, dtype=float)
cnt= 0
cc = 0

device = torch.device(f'cuda:{1}')
with torch.cuda.device(device):
    model = model.to('cuda:1')
   
    for dt in tqdm(loader_test):
        
        x, edge, y = dt.x.to('cuda:1'), dt.edge_index.to('cuda:1'), dt.y.to('cuda:1').long()
     
        output = model(x, edge)
        pred = torch.argmax(output, dim=-1)
      
        idxs = []
        for i in range(width):
            if pred[0][i] == y[0][i] and y[0][i] == 1:
                idxs.append(i)
      
        torch.cuda.empty_cache()
        explanation = explainer(x.squeeze(), edge)
        node_mask = explanation.node_mask
        
        if node_mask[idxs, :].shape != (0, features_count):
            node_mask = torch.mean(node_mask[idxs, :], dim=0)
            node_mask = np.array(node_mask.detach().cpu())
            mean_1 += node_mask
            cnt += 1

print('done interpretation')

100%|█████████████████████████████████████████████████████████████████████████████| 27121/27121 [36:48<00:00, 12.28it/s]

done interpretation
CPU times: user 32min 10s, sys: 4min 13s, total: 36min 23s
Wall time: 36min 48s


In [26]:
mean = mean_1 / cnt
print(mean.shape)

(1950,)


In [27]:
mean

array([-7.42047548e-04,  4.99803764e-03,  4.66836353e-03, ...,
        1.59512461e-18, -7.59804732e-11, -4.61208974e-08])

In [28]:
torch.save(torch.from_numpy(mean), 'GraphZSAGEConv_11L_IxG.pt')

# GNN_explainer

In [20]:
import captum
from captum.attr import IntegratedGradients, GradientShap, LayerGradCam, LRP
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [21]:
from torch_geometric.explain import Explainer, GNNExplainer

explainer = Explainer(
    model=wrappedmodel,
    algorithm=GNNExplainer(epochs=50),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='probs',
    ),
)
np.random.seed(42)
params = {'batch_size':1,
          'num_workers':4,
          'shuffle':True}

loader_train = DataLoader(train_dataset, **params)
loader_test = DataLoader(test_dataset, **params)

In [22]:
np.random.seed(42)
params = {'batch_size':1,
          'num_workers':4,
          'shuffle':True}

loader_train = DataLoader(train_dataset, **params)
loader_test = DataLoader(test_dataset, **params)

In [ ]:
%%time
features_count =1950
mean_1 = np.zeros(1950, dtype=float)
cnt= 0
cc = 0

device = torch.device(f'cuda:{1}')
with torch.cuda.device(device):
    model = model.to('cuda:1')
   
    for dt in tqdm(loader_test):
        
        x, edge, y = dt.x.to('cuda:1'), dt.edge_index.to('cuda:1'), dt.y.to('cuda:1').long()
     
        output = model(x, edge)
        pred = torch.argmax(output, dim=-1)
      
        idxs = []
        for i in range(width):
            if pred[0][i] == y[0][i] and y[0][i] == 1:
                idxs.append(i)
      
        torch.cuda.empty_cache()
        explanation = explainer(x.squeeze(), edge)
        node_mask = explanation.node_mask
        
        if node_mask[idxs, :].shape != (0, features_count):
            node_mask = torch.mean(node_mask[idxs, :], dim=0)
            node_mask = np.array(node_mask.detach().cpu())
            mean_1 += node_mask
            cnt += 1

print('done interpretation')

  2%|█▍                                                                          | 527/27121 [13:58<12:33:23,  1.70s/it]

In [ ]:
mean = mean_1 / cnt
print(mean.shape)


In [ ]:
mean

In [ ]:
torch.save(torch.from_numpy(mean), 'GraphZSAGEConv_11L_GNN_explainer.pt')